<a href="https://colab.research.google.com/github/erdemustun/Dosyalar/blob/main/KompozitNormalEmaKesisim_Tarama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Kompozit ve normal tarama,ortalamaları kesmişmi ona bakar, dosyaya kayıt edilir.

In [ ]:
!pip install git+https://github.com/rongardF/tvdatafeed tradingview-screener  backtesting tvDatafeed
!pip install retrying
#!pip install openpyxl
import numpy as np
import pandas as pd
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import Query, Column
import warnings
warnings.simplefilter(action='ignore')
tv = TvDatafeed()
from google.colab import files
from datetime import datetime
import traceback
from retrying import retry
import sys
from dataclasses import dataclass
from typing import List, Dict, Callable, Tuple

In [2]:
@retry(wait_fixed=2000, stop_max_attempt_number=3) # Hata olması halinde 2 sn bekleyip tekrar deneyecek, 3 kez deneyecek
def veri_getir(symbol,intervald):
    return tv.get_hist(symbol=symbol , exchange='BIST', interval=intervald, n_bars=250)

In [3]:
# günlük / haftalık mı çalışacak onun için
dt = datetime.now()
gun = dt.isoweekday()

if gun < 7:
    periyot = 'G'
elif gun == 7:
    periyot = 'H'
elif gun == 30 or gun == 31:
    periyot = 'A'
else:
    print("[Pazartesi-Cuma] ve Pazar günleri çalışır")
    sys.exit()

if periyot=='A':
    period='7y'
    intervald=Interval.in_monthly
    dosya='Aylik'
    freqs = 'M'
    offset = pd.DateOffset(months=1)  # bunu düzenle
elif periyot=='H':
    period='7y'
    intervald=Interval.in_weekly
    dosya='Haftalik'
    freqs = 'W'
    offset = pd.DateOffset(weeks=1)  # Haftalık frekans için bir hafta eklenir
elif periyot=='G':
    period='2y'
    intervald=Interval.in_daily
    dosya='Gunluk'
    freqs = 'D'
    offset = pd.DateOffset(days=1)  # Günlük frekans için bir gün eklenir
elif periyot=='S':
    period='1y'
    intervald=Interval.in_1_hour
    dosya='Saatlik'
elif periyot=='D':
    period='15d'
    intervald=Interval.in_15_minute
    dosya='15dakikalık'
else:
    print('Hatalı girdi')

In [4]:
piyasa= 'turkey'
#piyasa= 'germany'
#piyasa= 'america'
options = ['stock']  #['stock', 'fund','dr']

#Burada tupple dönüyor
HisseAd_Sorgu = (Query().set_markets(f'{piyasa}')
                .select('name', 'type')
                .where(Column('type').isin(options) #(['stock', 'fund'])
                    )
                .limit(20000) #deault değeri 50 olduğu için hepsini getirmiyor.Onu aşmak için yüksek değer koydum.
                .order_by('name')
                .get_scanner_data())[1]

# DataFrame'e dönüştür
HisseAd = pd.DataFrame(HisseAd_Sorgu, columns=['ticker', 'name', 'type'])
#len(HisseAd)
#HisseAd.to_csv('HisseAd.csv')

In [5]:
# Piyasa bazında filtreleme
if piyasa == 'america':
    # Amerika için NYSE veya NASDAQ ile başlayanları seç
    filtered_HisseAd = HisseAd[HisseAd['ticker'].str.startswith(('NYSE', 'NASDAQ'))]
elif piyasa == 'turkey':
    # Turkey için herhangi bir filtreleme uygulanmaz
    filtered_HisseAd = HisseAd
elif piyasa == 'germany':
    # Germany için özel filtreleme (örneğin XETRA ile başlayanlar)
    filtered_HisseAd = HisseAd[HisseAd['ticker'].str.startswith(('GETTEX', 'TRADEGATE', 'XETRA'))]
else:
    # Diğer piyasalar için de benzer filtrelemeler eklenebilir
    filtered_HisseAd = HisseAd

# Listeye dönüştür
Hisseler = filtered_HisseAd['ticker'].tolist()

# Sonuçları yazdır
#print(Hisseler)
#len(Hisseler)

In [6]:
# Dışarıdan eklemek istediğiniz endeks sembolleri
endeksler = [
    'XU030', 'XU100', 'XINSA', 'XBLSM', 'XUTEK', 'XGMYO',
    'XHOLD', 'XFINK', 'XSGRT', 'XBANK', 'XUMAL', 'XILTM',
    'XTRZM', 'XULAS', 'XELKT', 'XUHIZ', 'XMESY', 'XMANA',
    'XTAST', 'XKMYA', 'XKAGT', 'XTEKS', 'XGIDA', 'XUSIN', 'XMADN'
]

# Endeks sembollerini mevcut verilere ekleyin
Hisseler.extend(endeksler)
Hisseler = sorted(Hisseler)


In [ ]:
#Hisseler=Hisseler[:10]
# print(Hisseler)

In [28]:
@dataclass
class TechnicalIndicators:
    """Teknik gösterge hesaplamaları için sınıf"""

    @staticmethod
    def macd(src: pd.Series, fast: int = 12, slow: int = 26, signal: int = 9) -> tuple:
        """MACD hesaplama"""
        macd_line = src.ewm(span=fast, adjust=False).mean() - src.ewm(span=slow, adjust=False).mean()
        signal_line = macd_line.ewm(span=signal, adjust=False).mean()
        return macd_line, signal_line

    @staticmethod
    def mfi(data: pd.DataFrame, period: int = 14) -> pd.Series:
        """Money Flow Index hesaplama"""
        typical_price = (data['high'] + data['low'] + data['close']) / 3
        money_flow = typical_price * data['volume']

        price_diff = typical_price.diff()
        positive_flow = money_flow.where(price_diff > 0, 0)
        negative_flow = money_flow.where(price_diff < 0, 0)

        positive_mf = positive_flow.rolling(window=period).sum()
        negative_mf = negative_flow.rolling(window=period).sum()
        #mfi = 100 - (100 / (1 + positive_mf / negative_mf))
        return 100 - (100 / (1 + positive_mf / negative_mf))

    @staticmethod
    def rsi(data: pd.Series, period: int = 14) -> pd.Series:
        """RSI hesaplama,normal hesaptan farklı buradaki,buradaki fonk. aldım. """
        return data.rolling(window=period).apply(
            lambda x: 100 - (100 / (1 + (x.diff().clip(lower=0).sum() / x.diff().clip(upper=0).abs().sum())))
        )

class SignalGenerator:
    """Sinyal üretim sınıfı"""
    def __init__(self, ema_periods: List[int]):
        self.ema_periods = ema_periods
        self.indicators = TechnicalIndicators()

    def calculate_ema_signals(self, data: pd.DataFrame) -> pd.DataFrame:
        """EMA sinyallerini hesapla"""
        result = data.copy()

        # EMA hesaplamaları
        for period in self.ema_periods:
            result[f'ema_{period}'] = result['close'].ewm(span=period, adjust=False).mean()

        # Kesişim sinyalleri
        for period in self.ema_periods:
            col = f'ema_{period}'
            result[f'cX{col}'] = np.where(
                (result['close'].shift(1) < result[col].shift(1)) &
                (result['close'] > result[col]),
                '+',
                np.where(
                    (result['close'].shift(1) > result[col].shift(1)) &
                    (result['close'] < result[col]),
                    '-',
                    '0'
                )
            )

        # EMA kesişimlerini belirle
        result['EmaKesisim_Up'] = result[[c for c in result.columns if c.startswith('cXema_')]].apply(
            lambda row: ','.join([c.replace('cXema_', '') for c, val in row.items() if val == '+']),
            axis=1
        )
        result['EmaKesisim_Down'] = result[[c for c in result.columns if c.startswith('cXema_')]].apply(
            lambda row: ','.join([c.replace('cXema_', '') for c, val in row.items() if val == '-']),
            axis=1
        )

        # EMA ilişkilerini işle
        ema_columns = [f'ema_{period}' for period in self.ema_periods]
        result['EmaBuyuk'] = result[ema_columns].apply(
            lambda row: ','.join([str(period) for period, val in zip(self.ema_periods, row)
                                if result.loc[row.name, 'close'] > val]),
            axis=1
        )
        result['EmaKucuk'] = result[ema_columns].apply(
            lambda row: ','.join([str(period) for period, val in zip(self.ema_periods, row)
                                if result.loc[row.name, 'close'] < val]),
            axis=1
        )

        return result

    def generate_composite_signals(self, data: pd.DataFrame, reference_data: pd.DataFrame) -> pd.DataFrame:
        """Kompozit sinyalleri oluştur"""
        result = data.copy()

        # Kompozit hesaplama
        for column in ['open', 'high', 'low', 'close', 'volume']:
            result[column] = 100 * (data[column] / reference_data[column])

        return result

    @staticmethod
    def calculate_threshold_crosses(data: pd.Series, threshold: float = 50) -> pd.Series:
        """Eşik değer kesişimlerini hesapla"""
        up_cross = data.shift(1).lt(threshold) & data.ge(threshold)
        down_cross = data.shift(1).gt(threshold) & data.le(threshold)
        return up_cross.replace(True, '+').combine_first(down_cross.replace(True, '-')).fillna('0')

    @staticmethod
    def calculate_status(data: pd.Series, threshold: float = 50) -> pd.Series:
        """Durum hesapla"""
        return data.apply(lambda x: '+' if x > threshold else '-')

class TradingRules:
    """Ticaret kuralları sınıfı"""

    @staticmethod
    def kural1(data: pd.Series) -> bool:
        """Bütün ortalamalardan büyük olması"""
        return all(data['close'] > data[f'ema_{ema}'] for ema in [13, 21, 34, 55, 89, 144, 233])

    @staticmethod
    def kural2(data: pd.Series) -> bool:
        """Herhangi bir ortalamayı yukarı kesmiş"""
        return any(val.isdigit() for val in data['EmaKesisim_Up'].split(','))

    @staticmethod
    def kural3(data: pd.Series) -> bool:
        """RSI 50'yi yukarı kesmiş"""
        return data['RsiX50'] == '+'

    @staticmethod
    def kural4(data: pd.Series) -> bool:
        """RSI 50'den büyük"""
        return data['Rsi_Durum'] == '+'

    @staticmethod
    def kural5(data: pd.Series) -> bool:
        """MACD kesişim"""
        return data['Macd_K'] == '+'

    @staticmethod
    def kural6(data: pd.Series) -> bool:
        """MACD alımda"""
        return data['MACD_Status'] == '+'

    @staticmethod
    def kural7(data: pd.Series) -> bool:
        """MFI 50'yi kesmiş"""
        return data['MfiX50'] == '+'

    @staticmethod
    def kural8(data: pd.Series) -> bool:
        """MFI alımda"""
        return data['Mfi_Durum'] == '+'

class TradingSystem:
    """Geliştirilmiş ticaret sistemi"""
    def __init__(self, ema_periods: List[int]):
        self.signal_generator = SignalGenerator(ema_periods)
        self.rules = TradingRules()

    def evaluate_signals(self, data: pd.DataFrame) -> Dict[str, str]:
        """Sinyal değerlendirmesi"""
        latest = data.iloc[-1]

        return {
            "Sinyal1": '+' if (
                self.rules.kural2(latest) and
                self.rules.kural3(latest) and
                self.rules.kural5(latest)
            ) else '-',

            "Sinyal2": '+' if (
                self.rules.kural2(latest) and
                self.rules.kural4(latest) and
                self.rules.kural6(latest)
            ) else '-',

            "Sinyal3": '+' if (
                self.rules.kural2(latest) and
                self.rules.kural4(latest) and
                self.rules.kural5(latest)
            ) else '-',

            "Sinyal4": '+' if (
                self.rules.kural1(latest) and
                self.rules.kural4(latest) and
                self.rules.kural6(latest) and
                self.rules.kural8(latest)
            ) else '-'
        }

    def process_symbol_both(self, symbol_data: pd.DataFrame, reference_data: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """
        Hem göreceli hem de mutlak analiz yap
        Returns:
            Tuple[pd.DataFrame, pd.DataFrame]: (göreceli_sonuçlar, mutlak_sonuçlar)
        """
        try:
            # Tarih ve gerçek kapanış verilerini sakla
            real_close = symbol_data['close']
            real_change = (symbol_data['close'].pct_change()) * 100

            # Mutlak analiz (görecesiz)
            absolute_result = symbol_data.copy()
            absolute_result = self.calculate_all_indicators(absolute_result)

            # Göreceli analiz
            relative_result = self.signal_generator.generate_composite_signals(symbol_data, reference_data)
            relative_result = self.calculate_all_indicators(relative_result)

            # Göreceli analize gerçek kapanış ve değişim verilerini ekle
            relative_result['Kapanis'] = real_close
            relative_result['Deg%'] = real_change

            return relative_result, absolute_result

        except Exception as e:
            print(f"Hata oluştu: {e}")
            return None, None


    def calculate_all_indicators(self, df: pd.DataFrame) -> pd.DataFrame:
        """Tüm göstergeleri hesapla"""
        result = df.copy()

        # EMA hesaplamaları
        result = self.signal_generator.calculate_ema_signals(result)

        # Temel göstergeler
        result['Kapanis'] = df['close']
        result['Deg%'] = (df['close'].pct_change()) * 100

        # İndikatörler
        result['Macd'], result['Macd_signal'] = TechnicalIndicators.macd(result['close'])
        result['Rsi'] = TechnicalIndicators.rsi(result['close'])
        result['Mfi'] = TechnicalIndicators.mfi(result)

        # Eşik ve durum hesaplamaları
        result['RsiX50'] = self.signal_generator.calculate_threshold_crosses(result['Rsi'])
        result['Rsi_Durum'] = self.signal_generator.calculate_status(result['Rsi'])

        result['Macd_K'] = self.signal_generator.calculate_threshold_crosses(
            result['Macd'], threshold=result['Macd_signal'])
        result['MACD_Status'] = result.apply(
            lambda row: '+' if row['Macd'] > row['Macd_signal'] else '-', axis=1)

        result['MfiX50'] = self.signal_generator.calculate_threshold_crosses(result['Mfi'])
        result['Mfi_Durum'] = self.signal_generator.calculate_status(result['Mfi'])

        return result

def process_stocks_both(hisse_listesi: List[str], interval: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Hisse senetlerini hem göreceli hem de mutlak olarak analiz et
    Returns:
        Tuple[pd.DataFrame, pd.DataFrame]: (göreceli_sonuçlar, mutlak_sonuçlar)
    """
    trading_system = TradingSystem([13, 21, 34, 55, 89, 144, 233])
    relative_results = pd.DataFrame()
    absolute_results = pd.DataFrame()

    total_stocks = len(hisse_listesi)

    for i, hisse in enumerate(hisse_listesi, 1):
        try:
            print(f"\rİşleniyor: {hisse} ({i}/{total_stocks})", end='')

            # Verileri çek
            hisse_data = veri_getir(hisse, interval)
            reference_data = veri_getir('XU100', interval)


            # Her iki analizi yap
            relative_result, absolute_result = trading_system.process_symbol_both(hisse_data, reference_data)

            if relative_result is not None and absolute_result is not None:
                # Son satırları al
                relative_latest = relative_result.iloc[-1]
                absolute_latest = absolute_result.iloc[-1]

                # Sinyalleri hesapla ve ekle
                relative_signals = trading_system.evaluate_signals(relative_result)
                absolute_signals = trading_system.evaluate_signals(absolute_result)

                for key, value in relative_signals.items():
                    relative_latest[key] = value
                for key, value in absolute_signals.items():
                    absolute_latest[key] = value

                # Hisse ve tarih bilgilerini ekle
                relative_latest['Hisse'] = hisse
                absolute_latest['Hisse'] = hisse

                # datetime değerini al (orijinal veri setinden)
                relative_latest['Tarih'] = hisse_data.index[-1]
                absolute_latest['Tarih'] = hisse_data.index[-1]
                # if 'datetime' in hisse_data.columns:
                #     relative_latest['Tarih'] = hisse_data.iloc[-1]['datetime']
                #     absolute_latest['Tarih'] = hisse_data.iloc[-1]['datetime']

                # Sonuçları birleştir
                relative_results = pd.concat([relative_results, pd.DataFrame([relative_latest])], ignore_index=True)
                absolute_results = pd.concat([absolute_results, pd.DataFrame([absolute_latest])], ignore_index=True)

        except Exception as e:
            print(f"\n{hisse} için işlem başarısız: {e}")

    print("\nİşlem tamamlandı!")

    # Tarih ve hisse sütunlarını düzenle
    for df in [relative_results, absolute_results]:
        # Sütun isimlerini düzelt
        # if 'datetime' in df.columns:
        #     df = df.rename(columns={'datetime': 'Tarih'})
        # if 'symbol' in df.columns:
        #     df = df.rename(columns={'symbol': 'Hisse'})



        # # Tarih formatını düzenle
        # df['Tarih'] = pd.to_datetime(df['Tarih'], errors='coerce')
        # df['Tarih'] = df['Tarih'].dt.strftime("%d-%m-%y %H:%M")


        # Tarih formatını değiştirme
        df['Tarih'] = df['Tarih'].dt.strftime("%d-%m-%y")

        # BIST: ifadesini kaldırma
        df['Hisse'] = df['Hisse'].str.replace('BIST:', '')


    return relative_results, absolute_results


def save_results_to_excel(relative_df: pd.DataFrame, absolute_df: pd.DataFrame,
                         dosya_adi: str = "KompozitTarama") -> None:
    """Sonuçları Excel'e kaydet"""

    # Tarih formatı
    current_time = datetime.now().strftime("%d%m%y_%H:%M")

    # Sütun sıralaması
    siralama = [
        'Tarih', 'Hisse', 'Kapanis', 'Deg%', 'Sinyal1', 'Sinyal2', 'Sinyal3', 'Sinyal4',
        'EmaKesisim_Up', 'EmaKesisim_Down', 'EmaBuyuk', 'EmaKucuk', 'Macd',
        'Macd_signal', 'Rsi', 'RsiX50', 'Rsi_Durum', 'Macd_K', 'MACD_Status',
        'Mfi', 'MfiX50', 'Mfi_Durum', 'open', 'high', 'low', 'close', 'volume',
        'ema_13', 'ema_21', 'ema_34', 'ema_55', 'ema_89', 'ema_144', 'ema_233',
        'cXema_13', 'cXema_21', 'cXema_34', 'cXema_55', 'cXema_89', 'cXema_144', 'cXema_233'
    ]

    # Sayısal sütunları yuvarla
    for df in [relative_df, absolute_df]:
        numeric_cols = df.select_dtypes(include=['float', 'int']).columns
        df[numeric_cols] = df[numeric_cols].round(2)

    # Excel dosyasına kaydet
    with pd.ExcelWriter(f"{dosya_adi}_{current_time}.xlsx") as writer:
        relative_df.reindex(columns=siralama).to_excel(writer, sheet_name='Göreceli_Analiz', index=False)
        absolute_df.reindex(columns=siralama).to_excel(writer, sheet_name='Mutlak_Analiz', index=False)

    print(f"Veriler {dosya_adi}_{current_time}.xlsx dosyasına kaydedildi.")

In [ ]:
# Her iki analizi yap
goreceli_sonuclar, mutlak_sonuclar = process_stocks_both(Hisseler, intervald)

# Excel'e kaydet
save_results_to_excel(goreceli_sonuclar, mutlak_sonuclar, "KompozitTarama_Ema")

İşleniyor: BIST:BMSTL (88/570)